# RD-comparison of "Stereo" Transforms

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import sounddevice as sd
import math
import zlib

In [ ]:
def plot(x, y, xlabel='', ylabel='', title=''):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.grid()
    ax.xaxis.set_label_text(xlabel)
    ax.yaxis.set_label_text(ylabel)
    ax.plot(x, y, '.', markersize=1)
    plt.show(block=False)

## Produce some sound ... for example, say something during 5 s
This sound will be used to compare the RD performance of the transforms.

In [ ]:
fs = 44100
duration = 5.0  # seconds
x = sd.rec(int(duration * fs), samplerate=fs, channels=2, dtype=np.int16)
print("Say something!")
while sd.wait():
    pass
print("done")

In [ ]:
sd.play(x)
plot(np.linspace(0, len(x)-1, len(x)), x, "Time", "Amplitude", "Audio Signal")

In [ ]:
chunk = x[151000:152000]
plot(np.linspace(0, len(chunk)-1, len(chunk)), chunk, "Time", "Amplitude", "Audio Signal (zoom)")

In [ ]:
channel_0 = x[:, 0]
sd.play(channel_0)
plot(np.linspace(0, len(channel_0)-1, len(channel_0)), channel_0, "Time", "Amplitude", "Channel 0")

In [ ]:
channel_1 = x[:, 1]
sd.play(channel_1)
plot(np.linspace(0, len(channel_1)-1, len(channel_1)), channel_1, "Time", "Amplitude", "Channel 1")

In [ ]:
differences = channel_0 - channel_1

In [ ]:
sd.play(differences)
plot(np.linspace(0, len(differences)-1, len(differences)), differences, "Time", "Amplitude", "Channel 0 - Channel 1")

In [ ]:
channel_1 = channel_0 - differences

In [ ]:
recons_channel_1 = x[:, 1]
sd.play(recons_channel_1)
plot(np.linspace(0, len(recons_channel_1)-1, len(recons_channel_1)), recons_channel_1, "Time", "Amplitude", "Reconstructed Channel 1")

In [ ]:
channel_1.all() == recons_channel_1.all()

## RD stuff

In [ ]:
def average_energy(x):
    #return np.sum(x[:, 0].astype(np.double)*x[:, 0].astype(np.double))/len(x[:, 0]) + \
    #       np.sum(x[:, 1].astype(np.double)*x[:, 1].astype(np.double))/len(x[:, 1])
    return np.sum(x.astype(np.double)*x.astype(np.double))/len(x)
    
def RMSE(x, y):
    error_signal = x - y
    return math.sqrt(average_energy(error_signal))

# Based on https://stackoverflow.com/questions/15450192/fastest-way-to-compute-entropy-in-python
def entropy_in_bits_per_symbol(sequence_of_symbols):
    value, counts = np.unique(sequence_of_symbols, return_counts = True)
    probs = counts / len(sequence_of_symbols)
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    entropy = 0.
    for i in probs:
        entropy -= i * math.log(i, 2)

    return entropy

def deadzone_quantizer(x, quantization_step):
    k = (x / quantization_step).astype(np.int)
    return k

def deadzone_dequantizer(k, quantization_step):
    y = quantization_step * k
    return y

def deadzone_qdeq(x, quantization_step):
    k = deadzone_quantizer(x, quantization_step)
    y = deadzone_dequantizer(k, quantization_step)
    return k, y

def RD_curve(chunk, analyze, synthesize):
    RD_points = []
    for q_step in range(16, 1024, 32):
        analyzed_chunk = analyze(chunk)
        k, y = deadzone_qdeq(analyzed_chunk, q_step)
        rate = entropy_in_bits_per_symbol(k[:, 0]) + entropy_in_bits_per_symbol(k[:, 1])
        reconstructed_chunk = synthesize(y)
        distortion = RMSE(chunk, reconstructed_chunk)
        RD_points.append((rate, distortion))
    return RD_points

## [Karhunen-Loéve Transform (KLT)](https://en.wikipedia.org/wiki/Karhunen%E2%80%93Lo%C3%A8ve_theorem), [Hadamard](https://en.wikipedia.org/wiki/Hadamard_transform) and [Haar](https://en.wikipedia.org/wiki/Haar_wavelet#Haar_matrix)

The rows (i.e., the [basis](https://en.wikipedia.org/wiki/Basis_(linear_algebra)) vectors) of the of the [discrete](https://en.wikipedia.org/wiki/Discrete_transform) (all transforms discussed in this document are discrete) KLT consist of the [eigenvectors](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors#:~:text=In%20linear%20algebra%2C%20an%20eigenvector,which%20the%20eigenvector%20is%20scaled.) of the [autorrelation matrix](https://en.wikipedia.org/wiki/Autocorrelation#Auto-correlation_of_random_vectors) (also known as the [covariance](https://en.wikipedia.org/wiki/Covariance) matrix) of the transformed signal. In block-based transform coding (such as it happens in IterCom in which the audio signal is splitted into chunk) it can be assumed that, on average, the amount of information provided by a subband (the set of coefficients that correspond to a given frequency) is proportional to the variance in this subband.

The KLT minize the [geometric mean](https://en.wikipedia.org/wiki/Geometric_mean) of the variance of the transform coefficients. Hence, the KLT provides optimum energy concentration and therefore, the largest [transformation coding gain (TG)](/http://ws2.binghamton.edu/fowler/fowler%20personal%20page/EE523_files/Ch_13_3%20Transform%20Coding%20-%20Coding%20Gain%20&%20Classic%20Transforms%20(PPT).pdf) of any linear transform coding method. Unfortunately, the autocorrelation matrix depends on the transformed signal and therefore, the [change of basis](https://en.wikipedia.org/wiki/Change_of_basis) (the transform) provided by the KLT is signal dependent (each signal needs to be transformed by a different transform matrix), except in the case of 2-samples [block coding](https://web.stanford.edu/class/ee368b/Handouts/11-TransformCoding.pdf).

The TG is defined as the ratio of the arithmetic mean of the variances of the coefficients to their geometric means:

\begin{equation}
G_{TC} = \frac{\frac{1}{N}\sum_{i=1}^N{\sigma_i^2}}{(\prod_{i=1}^N\sigma_i^2)^{\frac{1}{N}}}
\end{equation}

In the context of signal compression, variance, energy, entropy and information are synonymous.

In [ ]:
def TCG(variances):
    N = len(variances)
    arithmetic_mean = np.sum(variances)/N
    geometric_mean = np.prod(variances)**(1/N)
    if geometric_mean == 0:
        geometric_mean = 1
    transform_coding_gain = arithmetic_mean/geometric_mean
    return transform_coding_gain

In [ ]:
# Forward transform:
#
#  [w[0]] = 1/sqrt(2) [1  1] [x[0]]
#  [w[1]]             [1 -1] [x[1]]
def KLT_analyze(x):
    w = np.empty_like(x, dtype=np.int32)
    w[:, 0] = np.rint((x[:, 0].astype(np.int32) + x[:, 1]) / math.sqrt(2)) # L
    w[:, 1] = np.rint((x[:, 0].astype(np.int32) - x[:, 1]) / math.sqrt(2)) # H
    return w

# Inverse transform:
#
#  [x[0]] = 1/sqrt(2) [1  1] [w[0]]
#  [x[1]]             [1 -1] [w[1]]
def KLT_synthesize(w):
    x = np.empty_like(w, dtype=np.int16)
    #x[:, 0] = np.rint((w[:, 0] + w[:, 1]) / math.sqrt(2)) # L(ow frequency subband)
    #x[:, 1] = np.rint((w[:, 0] - w[:, 1]) / math.sqrt(2)) # H(igh frequency subband)
    x[:, :] = KLT_analyze(w)
    return x

## [Mid/Side Transform (MST)](https://en.wikipedia.org/wiki/Joint_encoding#M/S_stereo_coding)

MST transforms the left and right channels into a *mid* channel and a *side* channel. The mid channel is the sum of the left and right channels, or $M=L+R$. The side channel is the difference of the left and right channels, or $S=L-R$. See also Section 2.2.1. *Sum-Difference coding* at this [PhD thesis](https://research.tue.nl/en/studentTheses/stereo-coding-by-two-channel-linear-prediction-and-rotation).

In [ ]:
# Forward transform:
#
#  [w[0]] = [1  1] [x[0]]
#  [w[1]]   [1 -1] [x[1]]

def MST_analyze(x):
    w = np.empty_like(x, dtype=np.int32)
    w[:, 0] = x[:, 0].astype(np.int32) + x[:, 1] # L(ow frequency subband)
    w[:, 1] = x[:, 0].astype(np.int32) - x[:, 1] # H(igh frequency subband)
    return w

# Inverse transform:
#
#  [x[0]] = 1/2 [1  1] [w[0]]
#  [x[1]]       [1 -1] [w[1]]

def MST_synthesize(w):
    x = np.empty_like(w, dtype=np.int16)
    x[:, 0] = (w[:, 0] + w[:, 1])/2 # L(ow frequency subband)
    x[:, 1] = (w[:, 0] - w[:, 1])/2 # H(igh frequency subband)
    return x

# This solution is slower than the previous one because: (1) an array of int32 is allocated,
# and (2) the memory locality is smaller (the division by 2 is not performed just when
# the samples are accesed).
def _MST_synthesize(w):
    x = np.empty_like(w, dtype=np.int16)
    x[:, :] = MST_analyze(w)
    x = x / 2
    return x

In [ ]:
x.shape

In [ ]:
w = MST_analyze(x)

In [ ]:
w.shape

In [ ]:
plot(np.linspace(0, len(w)-1, len(w)), w[:, 0], "Time", "Amplitude", "Mid Channel")
sd.play(w[:, 0].astype(np.int16))

In [ ]:
plot(np.linspace(0, len(w)-1, len(w)), w[:, 1], "Time", "Amplitude", "Side Channel")
sd.play(w[:, 1].astype(np.int16))

In [ ]:
plt.subplot(211)
plt.title('Spectrogram of the original chanels')
plt.specgram(x[:, 0], Fs=44100)
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.subplot(212)
plt.specgram(x[:, 1],Fs=44100)
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.subplot(211)
plt.title('Spectrogram of the Mid/Side channels')
plt.specgram(w[:, 0], Fs=44100)
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.subplot(212)
plt.specgram(w[:, 1], Fs=44100)
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()

In [ ]:
from scipy import signal
K1 = np.array([1.0, 1.0])
w1, h1 = signal.freqz(K1, fs=44100)
K2 = np.array([1.0, -1.0])
w2, h2 = signal.freqz(K2, fs=44100)

plt.subplot(211)
plt.title('Mid/Side filters frequency responses')
plt.plot(w1, 20 * np.log10(abs(h1)), 'b')
plt.xlabel('Frequency [rad/sample]')
plt.ylabel('Amplitude [dB]')
plt.subplot(212)
plt.plot(w2, 20 * np.log10(abs(h2)), 'b')
plt.xlabel('Frequency [rad/sample]')
plt.ylabel('Amplitude [dB]')
plt.show()

Notice that the cutoff frequency of these filters (Mid is a low-pass filter and Side is a high-pass filter) is very vage, in the sense that the transision between the pass band and the stop band is very wide. This implies that these filter are quite uneffective, as filters. However, this is the maximum performance that we can get using only two coeffients (or taps)/filter.

## MST vs KLT

In [ ]:
MST_RD_points = RD_curve(x, MST_analyze, MST_synthesize)

In [ ]:
KLT_RD_points = RD_curve(x, KLT_analyze, KLT_synthesize)

In [ ]:
plt.title("R/D comparative")
plt.xlabel("Bits per sample")
plt.ylabel("RMSE")
#plt.xscale("log")
#plt.yscale("log")
plt.scatter(*zip(*MST_RD_points), c='b', marker="+", label='MST')
plt.scatter(*zip(*KLT_RD_points), c='r', marker="x", label='KLT')
plt.legend(loc='upper right')
plt.show()

In [ ]:
%timeit MST_synthesize(MST_analyze(x))

In [ ]:
%timeit KLT_synthesize(KLT_analyze(x))

As it can be seen, MST and KLT are equivalent from a R/D perspective. However, MST is faster.

In [ ]:
LH = MST_analyze(x)
L = LH[:, 0]
H = LH[:, 1]
variances = [None]*2
variances[0] = np.var(L)
variances[1] = np.var(H)
print(TCG(variances))

In [ ]:
LH = KLT_analyze(x)
L = LH[:, 0]
H = LH[:, 1]
variances = [None]*2
variances[0] = np.var(L)
variances[1] = np.var(H)
print(TCG(variances))

The TCG of the KLT is sliiiightly better than the TCG of the MSC.

## S Transform (ST)
In-place?

In [ ]:
# https://www.researchgate.net/profile/Amir_Said2/publication/2598141_Reversible_Image_Compression_Via_Multiresolution_Representation_and_Predictive_Coding/links/56952cf008ae820ff074a536/Reversible-Image-Compression-Via-Multiresolution-Representation-and-Predictive-Coding.pdf
#
# Forward transform:
#
#  w[0] = ceil((x[0] + x[1])/2)
#  w[1] = x[0] - x[1] 
#
# Inverse transform:
#
#  x[0] = w[0] + ceil((w[1]+1)/2)
#  x[1] = x[0] - w[1]

def ST_analyze(x):
    w = np.empty_like(x, dtype=np.int32)
    w[:, 0] = np.ceil((x[:, 0].astype(np.int32) + x[:, 1])/2)
    w[:, 1] = x[:, 0].astype(np.int32) - x[:, 1]
    return w

def ST_synthesize(w):
    x = np.empty_like(w, dtype=np.int16)
    x[:, 0] = w[:, 0] + np.ceil((w[:, 1] + 1)/2)
    x[:, 1] = x[:, 0] - w[:, 1]
    return x

## ST vs MST

In [ ]:
ST_RD_points = RD_curve(x, ST_analyze, ST_synthesize)

In [ ]:
plt.title("R/D comparative")
plt.xlabel("Bits per sample")
plt.ylabel("RMSE")
#plt.xscale("log")
#plt.yscale("log")
plt.scatter(*zip(*MST_RD_points), c='b', marker="+", label='MSC')
plt.scatter(*zip(*ST_RD_points), c='r', marker="x", label='ST')
plt.legend(loc='upper right')
plt.show()

MST performs slighly better than ST.

In [ ]:
def stereo_DEFLATE_RD_curve(chunk, analyze, synthesize):
    RD_points = []
    for q_step in range(16, 1024, 32):
        analyzed_chunk = analyze(chunk)
        k, y = deadzone_qdeq(analyzed_chunk, q_step)
        subband_0 = k[:, 0].copy()
        subband_1 = k[:, 1].copy()
        rate = len(zlib.compress(subband_0)) + len(zlib.compress(subband_1))
        #rate = entropy_in_bits_per_symbol(k[:, 0]) + entropy_in_bits_per_symbol(k[:, 1])
        reconstructed_chunk = synthesize(y)
        distortion = RMSE(chunk, reconstructed_chunk)
        RD_points.append((rate, distortion))
    return RD_points

def _stereo_DEFLATE_RD_curve(chunk, analyze, synthesize):
    ch0_RD_points = []
    ch1_RD_points = []
    for q_step in range(16, 1024, 32):
        analyzed_chunk = analyze(chunk)
        k, y = deadzone_qdeq(analyzed_chunk, q_step)
        tmp = y.copy()
        y[:, 1] = 0
        
        reconstructed_chunk_with_subband_0 = synthesize(y)
        distortion_only_subband_0 = RMSE(chunk, reconstructed_chunk_with_subband_0)
        rate_subband_0 = len(zlib.compress(k[:, 0].copy()))
        ch0_RD_points.append((rate_subband_0, distortion_only_subband_0))
        
        y = tmp.copy()
        y[:, 0] = 0
        #y[:, 1] = 0
        reconstructed_chunk_with_subband_1 = synthesize(y)
        distortion_only_subband_1 = RMSE(chunk, reconstructed_chunk_with_subband_1)
        rate_subband_1 = len(zlib.compress(k[:, 1].copy()))
        ch1_RD_points.append((rate_subband_1, distortion_only_subband_1))
    return [ch0_RD_points, ch1_RD_points]

In [ ]:
MST_RD_points = stereo_DEFLATE_RD_curve(x, MST_analyze, MST_synthesize)

In [ ]:
plt.title("R/D comparative")
plt.xlabel("Bytes")
plt.ylabel("RMSE")
#plt.xscale("log")
#plt.yscale("log")
plt.scatter(*zip(*MST_RD_points), c='b', marker="+", label='MSC')
#plt.scatter(*zip(*KLT_RD_points), c='r', marker="x", label='KLT')
plt.legend(loc='upper right')
plt.show()

In [ ]:
MST_RD_points = _stereo_DEFLATE_RD_curve(x, MST_analyze, MST_synthesize)

In [ ]:
plt.title("RD comparative")
plt.xlabel("Bytes")
plt.ylabel("RMSE")
#plt.xscale("log")
#plt.yscale("log")
plt.scatter(*zip(*MST_RD_points[0]), c='b', marker="+", label='channel 0')
plt.scatter(*zip(*MST_RD_points[1]), c='r', marker="+", label='channel 1')
#plt.scatter(*zip(*KLT_RD_points), c='r', marker="x", label='KLT')
plt.legend(loc='upper right')
plt.show()

In [ ]:
mid_min_rate = min(MST_RD_points[0][:])[0]
mid_max_rate = max(MST_RD_points[0][:])[0]
mid_min_distortion = min(MST_RD_points[0][:])[1]
mid_max_distortion = max(MST_RD_points[0][:])[1]
print(mid_min_rate, mid_max_rate, mid_min_distortion, mid_max_distortion)

side_min_rate = min(MST_RD_points[1][:])[0]
side_max_rate = max(MST_RD_points[1][:])[0]
side_min_distortion = min(MST_RD_points[1][:])[1]
side_max_distortion = max(MST_RD_points[1][:])[1]
print(side_min_rate, side_max_rate, side_min_distortion, side_max_distortion)

#min_rate = min(mid_min_rate, side_min_rate)
max_rate = max(mid_max_rate, side_max_rate)


plt.subplot(121)
plt.scatter(*zip(*MST_RD_points[0]), c='b', marker="+", label='channel 0')
plt.title('Only Mid Channel')
plt.xlabel('Bytes')
plt.ylabel('RMSE')
#plt.xlim(0, max_rate)
plt.ylim(mid_max_distortion, mid_min_distortion)

plt.subplot(122)
plt.scatter(*zip(*MST_RD_points[1]), c='r', marker="+", label='channel 1')
plt.title('Only Side Channel')
plt.xlabel('Bytes')
#plt.xlim(0, max_rate)
#plt.ylim(4957, 5027)
#plt.ylabel('RMSE')

plt.show()

## Optimal quantization?

We compare two quantizations of the decomposition. First, $\Delta_1 = \Delta_2$. Next, $\Delta_1 = \sqrt{2}\Delta_2$.

In [ ]:
def stereo_DEFLATE_RD_curve_same_delta(chunk, analyze, synthesize):
    RD_points = []
    for q_step in range(16, 1024, 32):
        analyzed_chunk = analyze(chunk)
        k, y = deadzone_qdeq(analyzed_chunk, q_step)
        k_0 = k[:, 0].copy()
        k_1 = k[:, 1].copy()
        rate = len(zlib.compress(k_0)) + len(zlib.compress(k_1))
        #rate = entropy_in_bits_per_symbol(k[:, 0]) + entropy_in_bits_per_symbol(k[:, 1])
        #print(y)
        reconstructed_chunk = synthesize(y)
        distortion = RMSE(chunk, reconstructed_chunk)
        RD_points.append((rate, distortion))
        #RD_points.append((q_step, distortion))
    return RD_points

def stereo_DEFLATE_RD_curve_different_delta(chunk, analyze, synthesize):
    RD_points = []
    for q_step in range(16, 1024, 32):
        analyzed_chunk = analyze(chunk)
        subband_0 = analyzed_chunk[:, 0].copy()
        subband_1 = analyzed_chunk[:, 1].copy()
        #k_0, y_0 = deadzone_qdeq(subband_0, int(1000.0*q_step))
        k_0, y_0 = deadzone_qdeq(subband_0, q_step)
        k_1, y_1 = deadzone_qdeq(subband_1, int(1000.0*q_step))
        #k_1, y_1 = deadzone_qdeq(subband_1, q_step)
        rate = len(zlib.compress(k_0)) + len(zlib.compress(k_1))
        y = np.empty_like(x, dtype=np.int32)
        y[:, 0] = y_0
        y[:, 1] = y_1
        reconstructed_chunk = synthesize(y)
        distortion = RMSE(chunk, reconstructed_chunk)
        RD_points.append((rate, distortion))
    return RD_points

def stereo_DEFLATE_RD_curve_different_delta_b(chunk, analyze, synthesize):
    RD_points = []
    for q_step in range(16, 1024, 32):
        analyzed_chunk = analyze(chunk)
        subband_0 = analyzed_chunk[:, 0].copy()
        subband_1 = analyzed_chunk[:, 1].copy()
        k_0, y_0 = deadzone_qdeq(subband_0, 256)
        k_1, y_1 = deadzone_qdeq(subband_1, q_step)
        rate = len(zlib.compress(k_0)) + len(zlib.compress(k_1))
        y = np.empty_like(x, dtype=np.int32)
        y[:, 0] = y_0
        y[:, 1] = y_1
        reconstructed_chunk = synthesize(y)
        distortion = RMSE(chunk, reconstructed_chunk)
        RD_points.append((rate, distortion))
        #RD_points.append((q_step, distortion))
    return RD_points

def stereo_DEFLATE_RD_curve_different_delta_c(chunk, analyze, synthesize):
    RD_points = []
    analyzed_chunk = analyze(chunk)
    subband_0 = analyzed_chunk[:, 0].copy()
    subband_1 = analyzed_chunk[:, 1].copy()
    k_0, y_0 = deadzone_qdeq(subband_0, 256)
    k_1, y_1 = deadzone_qdeq(subband_1, 256)
    rate = len(zlib.compress(k_0)) + len(zlib.compress(k_1))
    y = np.empty_like(x, dtype=np.int32)
    y[:, 0] = y_0
    y[:, 1] = y_1
    reconstructed_chunk = synthesize(y)
    distortion = RMSE(chunk, reconstructed_chunk)
    RD_points.append((rate, distortion))
    return RD_points

In [ ]:
MST_RD_points_same_delta = stereo_DEFLATE_RD_curve_same_delta(x, MST_analyze, MST_synthesize)

In [ ]:
MST_RD_points_different_delta = stereo_DEFLATE_RD_curve_different_delta_b(x, MST_analyze, MST_synthesize)

In [ ]:
MST_RD_points_different_delta_c = stereo_DEFLATE_RD_curve_different_delta_c(x, MST_analyze, MST_synthesize)

In [ ]:
plt.title("R/D comparative")
plt.xlabel("Bits per sample")
plt.ylabel("RMSE")
#plt.xscale("log")
#plt.yscale("log")
plt.scatter(*zip(*MST_RD_points_same_delta), c='b', marker=".", label='$\Delta_1=\Delta_2$')
plt.scatter(*zip(*MST_RD_points_different_delta), c='r', marker=".", label='$\Delta_1=\sqrt{2}\Delta_2$')
plt.scatter(*zip(*MST_RD_points_different_delta_c), c='g', marker="o", label='$\Delta_1=\sqrt{2}\Delta_2$')
plt.legend(loc='upper right')
plt.show()

# ========================

## MSC/2

In [ ]:
# Forward transform:
#
#  [w[0]] = 1/2 [1  1] [x[0]]
#  [w[1]]       [1 -1] [x[1]]
#
# Inverse transform:
#
#  [x[0]] = [1  1] [w[0]]
#  [x[1]]   [1 -1] [w[1]]

def _MSC2_analyze(x):
    w = np.empty_like(x, dtype=np.int32)
    w[:, 0] = (x[:, 0].astype(np.int32) + x[:, 1])/2 # L(ow frequency subband)
    w[:, 1] = (x[:, 0].astype(np.int32) - x[:, 1])/2 # H(igh frequency subband)
    #w[:, 0] = w[:, 0] / 2
    return w

def MSC2_analyze(x):
    w = np.empty_like(x, dtype=np.int32)
    w[:, 0] = (x[:, 0].astype(np.int32) + x[:, 1])/2 # L(ow frequency subband)
    w[:, 1] = (x[:, 0].astype(np.int32) - x[:, 1])/2 # H(igh frequency subband)
    #w[:, 0] = w[:, 0] / 2
    return w

def _MSC2_synthesize(w):
    x = np.empty_like(w, dtype=np.int16)
    #w[:, 0] = w[:, 0] * 2
    x[:, :] = MSC2_analyze(w)
    return x

def MSC2_synthesize(w):
    x = np.empty_like(w, dtype=np.int16)
    x[:, 0] = w[:, 1] + x[:, 1]
    x[:, 1] = w[:, 0] - x[:, 0]
    #w[:, 0] = w[:, 0] * 2
    #x[:, :] = MSC2_analyze(w)
    return x

## MSC vs MSC2

In [ ]:
MSC2_RD_points = RD_curve(x, MSC2_analyze, MSC2_synthesize)

In [ ]:
plt.title("R/D comparative")
plt.xlabel("Bits per sample")
plt.ylabel("RMSE")
#plt.xscale("log")
#plt.yscale("log")
plt.scatter(*zip(*MSC_RD_points), c='b', marker="+", label='MSC')
plt.scatter(*zip(*MSC2_RD_points), c='r', marker="x", label='MSC2')
plt.legend(loc='upper right')
plt.show()

In [ ]:
%timeit MSC2_synthesize(MSC2_analyze(x))

## ?? transform

In [ ]:
# Forward transform:
#
#  [w[0]] = 1/2 [1  1] [x[0]]
#  [w[1]]       [1 -1] [x[1]]
#
# Inverse transform:
#
#  [x[0]] = [1  1] [w[0]]
#  [x[1]]   [1 -1] [w[1]]
#
# Forward transform:
#
#  w[1] = x[0] - x[1] 
#  w[0] = x[0] - w[1]/2
#  w[1] /= 2
#
# Inverse transform:
#
#  w[1] *= 2
#  x[0] = w[0] + w[1]/2
#  x[1] = x[0] - w[1]

def orig2_analyze(x):
    w = np.empty_like(x, dtype=np.int32)
    w[:, 1] = x[:, 0].astype(np.int32) - x[:, 1]
    w[:, 0] = x[:, 0] - w[:, 1] / 2
    w[:, 1] = w[:, 1] / 2
    return w

def orig2_synthesize(w):
    x = np.empty_like(w, dtype=np.int16)
    w[:, 1] = w[:, 1] * 2
    x[:, 0] = w[:, 0] + w[:, 1]/2
    x[:, 1] = x[:, 0] - w[:, 1]
    return x

In [ ]:
orig2_RD_points = RD_curve(x, orig2_analyze, orig2_synthesize)

In [ ]:
plt.title("R/D comparative")
plt.xlabel("Bits per sample")
plt.ylabel("RMSE")
#plt.xscale("log")
#plt.yscale("log")
plt.scatter(*zip(*MSC_RD_points), c='b', marker="+", label='MSC')
plt.scatter(*zip(*orig2_RD_points), c='r', marker="x", label='?')
plt.legend(loc='upper right')
plt.show()

In [ ]:
%timeit orig2_synthesize(orig2_analyze(x))

## ? transform

In [ ]:
# Forward transform:
#
#  w[1] = x[1] - x[0] 
#  w[0] = x[0] + w[1]/2
#  w[1] /= 2
#
# Inverse transform:
#
#  w[1] *= 2
#  x[0] = w[0] - w[1]/2
#  x[1] = x[0] + w[1]

def orig_analyze(x):
    w = np.empty_like(x, dtype=np.int32)
    w[:, 1] = x[:, 1].astype(np.int32) - x[:, 0]
    w[:, 0] = x[:, 0] + np.rint(w[:, 1] / 2)
    w[:, 1] = np.rint(w[:, 1] / 2)
    return w

def orig_synthesize(w):
    x = np.empty_like(w, dtype=np.int16)
    w[:, 1] = w[:, 1] * 2
    x[:, 0] = w[:, 0] - np.rint(w[:, 1]/2)
    x[:, 1] = w[:, 1] + x[:, 0]
    return x

In [ ]:
orig_RD_points = RD_curve(x, orig_analyze, orig_synthesize)

In [ ]:
plt.title("R/D comparative")
plt.xlabel("Bits per sample")
plt.ylabel("RMSE")
#plt.xscale("log")
#plt.yscale("log")
plt.scatter(*zip(*MSC_RD_points), c='b', marker="+", label='MSC')
plt.scatter(*zip(*orig_RD_points), c='r', marker="x", label='?')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Introduction to Data Compression (Sayood), pag. 402

def pordeterminar_analyze(chunk):
    analyzed_chunk = np.empty_like(chunk, dtype=np.int16)
    analyzed_chunk[:, 1] = chunk[:, 1] - chunk[:, 0]
    analyzed_chunk[:, 0] = chunk[:, 0] + np.rint(analyzed_chunk[:, 1]/2)
    return analyzed_chunk

def pordeterminar_synthesize(analyzed_chunk):
    chunk = np.empty_like(analyzed_chunk, dtype=np.int16)
    chunk[:, 0] = analyzed_chunk[:, 0] - np.rint(analyzed_chunk[:, 1]/2)
    chunk[:, 1] = analyzed_chunk[:, 1] + chunk[:, 0]
    return chunk

In [ ]:
def orig2_analyze(chunk):
    analyzed_chunk = np.empty_like(chunk, dtype=np.int16)
    analyzed_chunk[:, 1] = chunk[:, 1] - chunk[:, 0]
    analyzed_chunk[:, 0] = (chunk[:, 0] + chunk[:, 1]) // 2
    return analyzed_chunk

def orig2_synthesize(analyzed_chunk):
    chunk = np.empty_like(analyzed_chunk, dtype=np.int16)
    chunk[:, 1] = analyzed_chunk[:, 0] + analyzed_chunk[:, 1] // 2
    chunk[:, 0] = chunk[:, 1] - analyzed_chunk[:, 1]
    return chunk

In [ ]:
def orig3_analyze(chunk):
    analyzed_chunk = np.empty_like(chunk, dtype=np.int16)
    analyzed_chunk[:, 1] = chunk[:, 1] - chunk[:, 0]
    analyzed_chunk[:, 0] = chunk[:, 0] + np.rint(analyzed_chunk[:, 1]/2)
    analyzed_chunk[:, 1] = analyzed_chunk[:, 1] // 2
    return analyzed_chunk

def orig3_synthesize(analyzed_chunk):
    chunk = np.empty_like(analyzed_chunk, dtype=np.int16)
    analyzed_chunk[:, 1] = analyzed_chunk[:, 1] * 2
    chunk[:, 0] = analyzed_chunk[:, 0] - np.rint(analyzed_chunk[:, 1]/2)
    chunk[:, 1] = analyzed_chunk[:, 1] + chunk[:, 0]
    return chunk

In [ ]:
orig_RD_points = RD_curve(x, orig_analyze, orig_synthesize)

In [ ]:
orig2_RD_points = RD_curve(x, orig2_analyze, orig2_synthesize)

In [ ]:
orig3_RD_points = RD_curve(x, orig3_analyze, orig3_synthesize)

In [ ]:
pd_RD_points = RD_curve(x, pordeterminar_analyze, pordeterminar_synthesize)

In [ ]:
plt.title("R/D comparative")
plt.xlabel("Bits per sample")
plt.ylabel("RMSE")
#plt.xscale("log")
#plt.yscale("log")
plt.scatter(*zip(*MS_RD_points), c='b', marker="o", label='MS')
plt.scatter(*zip(*KLT_RD_points), c='g', marker="^", label='KLT')
plt.scatter(*zip(*orig_RD_points), c='r', marker="v", label='orig')
plt.scatter(*zip(*orig2_RD_points), c='m', marker="x", label='orig2')
plt.scatter(*zip(*orig3_RD_points), c='c', marker="h", label='orig3')
plt.scatter(*zip(*pd_RD_points), c='y', marker="+", label='pd')
plt.legend(loc='upper right')
plt.show()